In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

sys.path.append("/home/bongo/porter_notebook/research/qwen3")

# config 및 file_managing에서 필요한 변수/함수 임포트
from config import AGD20K_PATH, model_name
from file_managing import make_input_image

# ------------------------------------------------------
# 1. 모델 로딩
# ------------------------------------------------------
# SDPA를 끄면 메모리를 좀 더 쓰지만 Attention Map은 확실히 나옵니다.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_ENABLE_SDPA"] = "0"  # 1 -> 0 변경
os.environ["PYTORCH_ENABLE_FLASH_SDP"] = "0"
os.environ["PYTORCH_ENABLE_MEM_EFFICIENT_SDP"] = "0"

print(f"🤖 Loading {model_name} for selection...")
model = Qwen3VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    device_map="cuda", 
)

processor = AutoProcessor.from_pretrained(model_name)
device = model.device

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!


🤖 Loading Qwen/Qwen3-VL-2B-Instruct for selection...


In [2]:
image_path, target_action, object_name = "/home/DATA/AGD20K/Seen/trainset/exocentric/carry/surfboard/carry_surfboard_001939.jpg", "carry", "surfboard"

In [3]:

# -----------------------------------------------------------
# 1. 전처리: 핵심 동사 추출
# -----------------------------------------------------------
# 사전 없이 단순 split만 사용
core_action = target_action.split('_')[0].lower() 

# -----------------------------------------------------------
# 2. 질문: 동사 나열 유도
# -----------------------------------------------------------
query = f"What actions is the person doing with the {object_name}? list all the possible verbs. Only list the verbs."


image_base64 = make_input_image(str(image_path))


messages = [
    {"role": "user", "content": [
        {"type": "image", "image": f"data:image/jpeg;base64,{image_base64}"},
        {"type": "text", "text": query}
    ]}
]
inputs = processor.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_dict=True, return_tensors="pt").to(model.device)

# Vision Token 인덱스 찾기
input_ids_list = inputs.input_ids[0].tolist()
vis_start_id = processor.tokenizer.convert_tokens_to_ids("<|vision_start|>")
vis_end_id = processor.tokenizer.convert_tokens_to_ids("<|vision_end|>")

vis_start_idx = input_ids_list.index(vis_start_id)
vis_end_idx = input_ids_list.index(vis_end_id)

with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=30, output_attentions=True, return_dict_in_generate=True)
    
output_ids = generated_ids.sequences[0][inputs.input_ids.shape[1]:]
full_text = processor.decode(output_ids, skip_special_tokens=True).lower()
full_text

'carry'

In [4]:
output_ids

tensor([ 73765, 151645], device='cuda:0')

In [14]:


# -----------------------------------------------------------
# 5. Targeted Scoring (Safety Net 추가)
# -----------------------------------------------------------
total_vis_score = 0.0     # 매칭된 토큰들의 합
matched_count = 0         # 매칭된 토큰 수

all_tokens_energy_sum = 0.0 # (Safety Net용) 전체 토큰 에너지 합
valid_token_count = 0       # (Safety Net용) 전체 유효 토큰 수

for i, token_id in enumerate(output_ids):
    token_str = processor.decode([token_id], skip_special_tokens=True).lower().strip()
    
    if not token_str: continue # 공백 토큰 무시

    # --- [에너지 계산] ---
    # 매칭 여부와 상관없이 일단 현재 토큰의 에너지를 계산해둡니다.
    token_energy = 0.0
    # ... 인덱스 찾는 부분 직후 ...
    print(f"🔍 [Debug Indices] Start: {vis_start_idx}, End: {vis_end_idx}")
    print(f"🔍 [Debug Length] Image Tokens Count: {vis_end_idx - vis_start_idx - 1}")

    for layer_attn in generated_ids.attentions[i]:
        # Vision Token 영역만 슬라이싱하여 합산
        vision_attn = layer_attn[0, :, 0, vis_start_idx+1 : vis_end_idx]
        token_energy += vision_attn.float().sum().item()
        # print(f"token_energy : {token_energy}")
    
    # Safety Net을 위해 전체 누적
    all_tokens_energy_sum += token_energy
    valid_token_count += 1

    # --- [토큰 매칭 확인] ---
    # 1. Core Action이 Token을 포함 (예: carry >= car) -> BPE 파편화 대응
    # 2. Token이 Core Action을 포함 (예: carrying >= carry) -> 변형 대응
    if (core_action in token_str) or (token_str in core_action and len(token_str) > 1): 
        # print("good")
        # len > 1 조건: 'c', 'a' 같은 너무 짧은 파편이 엄한 단어에 매칭되는 것 방지
        total_vis_score += token_energy
        matched_count += 1
        # print(a)
    print(f"Token {i}: {token_str}  / :{total_vis_score}")
# -----------------------------------------------------------
# [결과 반환 로직]
# -----------------------------------------------------------
if matched_count > 0:
    # 1. 정확히 매칭된 토큰이 있으면 그 점수 사용 (Best)
    final_score = total_vis_score / matched_count
    print("FIN :",  final_score, full_text )
else:
    # 2. [Safety Net] 매칭된 토큰은 없지만, full_text에는 정답이 있었음!
    # 토크나이징 문제로 판단하고, 전체 문장의 평균 에너지를 반환 (Fallback)
    if valid_token_count > 0:
        final_score = all_tokens_energy_sum / valid_token_count
        print("FIN2 :",  final_score, full_text )
    else:
        print("FIN3 :",  0.0, full_text )# 토큰이 없으면 0점

print( final_score, full_text)


🔍 [Debug Indices] Start: 3, End: 965
🔍 [Debug Length] Image Tokens Count: 961
Token 0: carry  / :0.0
FIN : 0.0 carry
0.0 carry


In [13]:
layer_attn[0, :, 0, : ]

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.bfloat16)

In [7]:
print(f"🔍 Current Attention Implementation: {model.config}")

🔍 Current Attention Implementation: Qwen3VLConfig {
  "architectures": [
    "Qwen3VLForConditionalGeneration"
  ],
  "dtype": "bfloat16",
  "image_token_id": 151655,
  "model_type": "qwen3_vl",
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 2048,
    "initializer_range": 0.02,
    "intermediate_size": 6144,
    "max_position_embeddings": 262144,
    "model_type": "qwen3_vl_text",
    "num_attention_heads": 16,
    "num_hidden_layers": 28,
    "num_key_value_heads": 8,
    "rms_norm_eps": 1e-06,
    "rope_scaling": {
      "mrope_interleaved": true,
      "mrope_section": [
        24,
        20,
        20
      ],
      "rope_type": "default"
    },
    "rope_theta": 5000000,
    "tie_word_embeddings": true,
    "use_cache": true,
    "vocab_size": 151936
  },
  "tie_word_embeddings": true,
  "transformers_v

In [6]:
import numpy as np

In [12]:
layer_attn[0, :, 0, :].sum()

tensor(16., device='cuda:0', dtype=torch.bfloat16)

In [10]:
np.shape(layer_attn[0, :, 0, :])

torch.Size([16, 993])

In [9]:
token_energy

0.0